### This is my first (public) notebook of playing with pyspark

The goal of this notebook is to demonstrare a simple use case of:
1. using pyspark (python wrapper for spark) and access existing spark cluster
2. loading data into spark
3. accessing the data using the rdd-api, dataframe-api, and dataset-api
4. simple data exploration

I'll be using a sample of the gutenberg project - https://web.eecs.umich.edu/~lahiri/gutenberg_dataset.html

#### The settings I'm using here:
- spark: version 2.0.0
- python: 3.5

In [1]:
#import urllib3 #ability to access data by url
import pyspark #python wrapper for spark
sc.stop()
sc = pyspark.SparkContext(appName="spark-project")

In [2]:
rdd = sc.textFile("s3n://ak-public-sandbox/datasets/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(5)

['The Project Gutenberg EBook of Told in a French Garden, by Mildred Aldrich',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included']

In [3]:
#Let's see some fitering capabilities
rdd.filter(lambda line: "mankind" in line).count()

46

In [6]:
#the good old word count
counts = rdd.flatMap(lambda line: line.split(" ")).map(lambda word: (word,1)).reduceByKey(lambda a,b: a + b)
counts.collect()

AttributeError: 'list' object has no attribute 'flatMap'

In [ ]:
rdd.filter(lambda line: "mankind" in line).saveAsTextFile("s3n://spark-course-data2/result/lesson_2-2")

In [ ]:
mankindLines = sc.textFile("s3n://spark-course-data2/result/lesson_2-2")
mankindLines.count()

In [ ]:
rdd = sc.wholeTextFiles("s3n://spark-course-data2/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(1)

In [5]:
keys = rdd.map( lambda x:x[0] )
keys.count()
#keys.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 4 times, most recent failure: Lost task 0.3 in stage 5.0 (TID 37, 172.31.16.74): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/root/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/root/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/root/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/root/spark/python

In [ ]:
import boto3
session = boto3.Session() # leave out the profile_name argument if you haven't defined profiles
s3 = boto3.resource('s3')
bucket = s3.Bucket('spark-course-data2')
objs = bucket.objects.filter(Prefix='gutenberg_dataset/1')
for obj in objs:  
  print(obj.key)

In [ ]:
obj = s3.Object('spark-course-data2', 'gutenberg_dataset/1/9/9/1997/1997.txt')
content = obj.get()["Body"].read().decode("utf-8")
print(content[:100])

In [ ]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://www.kaggle.com/c/titanic/download/train.csv')
r.status
#response = urllib3.urlopen('')
#html = response.read()